In [27]:
# Importing the necessary libraries/modules
import numpy as np
import time
import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # Import the Axes3D module explicitly
from sklearn import neighbors
from astropy.table import Table, Column
from IPython.display import display

In [28]:
#Vast libraries/modules
from vast.voidfinder.distance import z_to_comoving_dist
from vast.voidfinder import ra_dec_to_xyz
from vast.voidfinder._voidfinder_cython_find_next import MaskChecker
from vast.voidfinder.voidfinder_functions import xyz_to_radecz

In [29]:
## File names

'NSA_main_mask.pickle'
"table1_VF_MaximalSpheres_2Cosmologies.dat.fits"
"table2_VF_AllSpheres_2Cosmologies.dat.gz.fits"
"table3_V2_Zobovoids_2Prunings_2Cosmologies.dat.gz.fits"
"table4_V2_Zones_2Prunings_2Cosmologies.dat.fits"
"table5_V2_Galaxies_2Prunings_2Cosmologies.dat.gz.fits"


'table5_V2_Galaxies_2Prunings_2Cosmologies.dat.gz.fits'

In [57]:
def transpose_table(table_before, id_col_name='ID'):
    new_column_names = tuple(table_before[table_before.colnames[0]])
    new_row_names = table_before.colnames[1:]
    
    table_after = Table(names=new_column_names)
    
    for r in new_row_names:
        # Skip the first row ('float64')
        if r != 'float64':
            table_after.add_row(table_before[r])
    
    if id_col_name != '':
        table_after.add_column(Column(new_row_names, name=id_col_name), index=0)
    
    return table_after



In [31]:
def load_mask(mask_file):
    with open(mask_file, "rb") as f:
        mask, mask_resolution, dist_limits = pickle.load(f)
    return mask, mask_resolution, dist_limits

In [32]:
def filter_and_analyze(table, desired_cosmo, desired_prune=None):
    # Filter based on the 'Cosmo' value
    filtered_table = table[table['Cosmo'] == desired_cosmo]

    # If the table has a 'Prune' column, filter based on the 'Prune' value
    if 'Prune' in table.colnames and desired_prune is not None:
        filtered_table = filtered_table[filtered_table['Prune'] == desired_prune]

    return filtered_table

In [33]:

def calc_volume_boundaries(void_cat_A, void_cat_B):
    """Compute the boundaries of the minimal rectangular volume (parallelpiped)
    that completely contains two void catalogs.
   
    Parameters
    ----------
    void_cat_A : astropy.Table
        Table of void data from the first catalog.
    void_cat_B : astropy.Table
        Table of void data from the second catalog.
       
    Returns
    -------
    x_min : float
        Minimum x-coordinate of the enclosing volume.
    x_max : float
        Maximum x-coordinate of the enclosing volume.
    y_min : float
        Minimum y-coordinate of the enclosing volume.
    y_max : float
        Maximum y-coordinate of the enclosing volume.
    z_min : float
        Minimum z-coordinate of the enclosing volume.
    z_max : float
        Maximum z-coordinate of the enclosing volume.
    """
   
    # Calculating the minimum and maximum x-coordinates
    x_min = np.minimum(np.min(void_cat_A['x']), np.min(void_cat_B['x']))
    x_max = np.maximum(np.max(void_cat_A['x']), np.max(void_cat_B['x']))
   
    # ' ' y-coordinates
    y_min = np.minimum(np.min(void_cat_A['y']), np.min(void_cat_B['y']))
    y_max = np.maximum(np.max(void_cat_A['y']), np.max(void_cat_B['y']))

    # ' ' z-coordinates
    z_min = np.minimum(np.min(void_cat_A['z']), np.min(void_cat_B['z']))
    z_max = np.maximum(np.max(void_cat_A['z']), np.max(void_cat_B['z']))
    
    # Returning the boundaries
    return x_min, x_max, y_min, y_max, z_min, z_max

In [34]:

def generate_grid_points(x_min, x_max, y_min, y_max, z_min, z_max, spacing=1.0):
    """Creates a dense rectangular grid of points in 3D for the void volume calculation.
   
    Parameters
    ----------
    x_min : float
        Minimum x-coordinate of the grid boundaries.
    x_max : float
        Maximum x-coordinate of the grid boundaries.
    y_min : float
        Minimum y-coordinate of the grid boundaries.
    y_max : float
        Maximum y-coordinate of the grid boundaries.
    z_min : float
        Minimum z-coordinate of the grid boundaries.
    z_max : float
        Maximum z-coordinate of the grid boundaries.
        
    Default spacing is set at 1.0 Mpc
   
    Returns
    -------
    point_coords : numpy.ndarray with shape (3, N)
        The grid coordinates where N is the number of grid points.
    """
   
    x_range = np.arange(x_min, x_max, spacing)
    y_range = np.arange(y_min, y_max, spacing)
    z_range = np.arange(z_min, z_max, spacing)
    
    X, Y, Z = np.meshgrid(x_range, y_range, z_range, indexing='ij')
    grid_points = np.column_stack((X.ravel(), Y.ravel(), Z.ravel())).T
    
    return grid_points

In [35]:
def plot_data(data_table_V1, data_table_V1max, title):
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(projection='3d')

    ax.scatter(data_table_V1['x'], data_table_V1['y'], data_table_V1['z'],
               color='maroon', s=0.1, alpha=0.5,
               label='holes')
    ax.scatter(data_table_V1max['x'], data_table_V1max['y'], data_table_V1max['z'],
               color='navy', s=10, alpha=1.0,
               label='max')
    ax.set(xlabel='X [Mpc/h]', ylabel='Y [Mpc/h]', zlabel='Z [Mpc/h]')

    ax.legend(loc='upper right', fontsize=10)

    plt.title(title)
    plt.show()

In [36]:
def plot_3d_points(pts):
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(projection='3d')
    ax.scatter(pts[:, 0], pts[:, 1], pts[:, 2], color='blue', s=1, alpha=0.5)
    ax.set(xlabel='X [Mpc/h]', ylabel='Y [Mpc/h]', zlabel='Z [Mpc/h]')
    plt.title("Points from Void Volume")
    plt.show()

In [37]:
def points_in_mask_filter(pts, mask, mask_resolution, rmin, rmax):
    """Determines which grid points generated by generate_grid_points lie in the survey mask

    Parameters
    ----------
    pts : numpy.ndarray with shape (N, 3)
        The grid points generated by generate_grid_points.
    mask : numpy.ndarray with shape (N, M)
        The survey mask returned by vast.voidfinder.multizmask.generate_mask.
    mask_resolution : int
        The survey mask resolution returned by vast.voidfinder.multizmask.generate_mask.
    rmin : float
        The comoving minimum distance within the survey volume.
    rmax : float
        The comoving maximum distance within the survey volume.

    Returns
    -------
    points_in_mask : numpy.ndarray with shape (3, P)
        The list of grid points within the survey mask, where P is the number of points in the mask.
    """
    
    # Ensure pts has shape (N, 3)
    if pts.shape[1] != 3:
        pts = pts.T  # Transpose if the shape is (3, N)

    # Initializing a boolean array to tag points in the mask
    points_boolean = np.ones(pts.shape[0], dtype=bool)

    # Declaring a MaskChecker object from VAST
    mask_checker = MaskChecker(0, mask, mask_resolution, rmin, rmax)

    # Flagging the points that end up outside the mask
    for i in range(pts.shape[0]):
        # Get the current point
        curr_pt = pts[i, :]

        # Checking if the point is not in the mask
        not_in_mask = mask_checker.not_in_mask(curr_pt)

        # Invert not_in_mask to tag points in the mask
        points_boolean[i] = not bool(not_in_mask)

    # Now selecting all points that did not get filtered out during the above process
    points_in_mask = pts[points_boolean, :]

    return points_in_mask


In [38]:
def kd_tree(void_cat):
    """Creates a KD-Tree from the x-y-z coordinates of a void catalog.
   
    Parameters
    ----------
    void_cat : astropy.Table
        The given void catalog, which includes columns 'x', 'y', and 'z'.
   
    Returns
    -------
    sphere_tree : sklearn.neighbors._kd_tree.KDTree
        The KD-Tree object for the void catalog.
    """
    # Extracting x, y, and z coordinates from the void catalog
    cx = void_cat['x']
    cy = void_cat['y']
    cz = void_cat['z']

    # Formatting the void centers in an array of shape (N, 3)
    sphere_coords = np.array([cx, cy, cz]).T

    # Creating a KD-Tree from the coordinates using scikit-learn's neighbors module
    sphere_tree = neighbors.KDTree(sphere_coords)
   
    return sphere_tree

In [39]:
def point_query_VF(point_coords, sphere_tree, void_catalog):

    dist, idx = sphere_tree.query(point_coords, k = 1)

    true_inside = dist < void_catalog['Rad'][idx]

    return true_inside

In [40]:
def prep_V2_cat(V2_galzones, V2_zonevoids, galaxy_fits):
    """Formats a V2 catalog
   
    Parameters
    ----------
    V2_galzones: Astropy Table
        The V2 galzones catalogue
    V2_zonevoids: Astropy Table
        The V2 zonevoids catalogue
    data_table_vl: Astropy Table
        The galaxy catalog used to genreate the V2 void catalog
    Returns
    -------
    V2_galzones : Astropy table
        The V2_galzones object formatted for use with teh volume overlap caculator
    """
   
    # TODO: remove hardcoding of these values and allow user to set them
    omega_M = np.float32(0.3)
    h = np.float32(1.0)
   
    # Sort the catalogs
    galaxy_fits.sort('NSAID')
    V2_galzones.sort('NSAID')
   
    # Select only galaxies with positive redshift
    z_boolean = galaxy_fits['Z']>0
    galaxy_fits = galaxy_fits[z_boolean]
    
    # Get unique NSAID values from V2_galzones
    unique_nsaids_v2 = set(V2_galzones['NSAID'])

    galaxy_nsaids = np.array(galaxy_fits['NSAID'])

    # Filter galaxy_fits based on unique NSAID values from V2_galzones
    filtered = np.isin(galaxy_nsaids, list(unique_nsaids_v2))
    filtered_galaxy_fits = galaxy_fits[filtered]

    # Get unique NSAID values from galaxy_fits
    unique_nsaids_galaxy = set(filtered_galaxy_fits['NSAID'])

    # Filter V2_galzones based on unique NSAID values from galaxy_fits
    filtered_v = np.isin(V2_galzones['NSAID'], list(unique_nsaids_galaxy))
    filtered_v2_galzones = V2_galzones[filtered_v]


    # Make sure that each galaxy in the catalog has a zone in the galzones file
    assert len(filtered_galaxy_fits) == len(filtered_v2_galzones)
   
    # Calculate galaxy comoving distances
    Rgal = z_to_comoving_dist(filtered_galaxy_fits['Z'].astype(np.float32),omega_M,h)
    filtered_galaxy_fits['Rgal'] = Rgal
    
    # Rename 'RA' and 'DEC' columns to lowercase 'ra' and 'dec' inside filtered_galaxy_fits
    filtered_galaxy_fits.rename_column('RA', 'ra')
    filtered_galaxy_fits.rename_column('DEC', 'dec')
   
    # Calculate the xyz coordinates
    galaxies_xyz = ra_dec_to_xyz(filtered_galaxy_fits)
    filtered_v2_galzones['x'] = galaxies_xyz[:,0]
    filtered_v2_galzones['y'] = galaxies_xyz[:,1]
    filtered_v2_galzones['z'] = galaxies_xyz[:,2]
    
    filtered_v2_galzones.sort('zone')
    V2_zonevoids.sort('zone')

    # Mark zones that belong to voids
    filtered_v2_galzones['in_void'] = V2_zonevoids[filtered_v2_galzones['zone']]['void1'] > -1

    # Sort the table by NSAID
    filtered_v2_galzones.sort('NSAID')

    return filtered_v2_galzones

In [41]:
def point_query_V2(point_coords, sphere_tree, void_cat):
    """Determines which members of a set of grid points are inside V2 voids
   
    Parameters
    ----------
    point_coords : numpy.ndarray with shape (3, N)
        The grid points within the survey volume.
    sphere_tree : sklearn.neighbors._kd_tree.KDTree
        The KD-Tree object for the void catalog generated by kd_tree.
    void_cat : astropy.Table
        The V2 void catalog which includes a void radius column 'radius'.
   
    Returns
    -------
    true_inside : numpy.ndarray with shape (N,)
        Indicates which grid points are inside voids.
    """
   
    idx = sphere_tree.query(point_coords, k=1, return_distance=False)
   
    true_inside = void_cat[idx]['in_void']

    return true_inside


In [42]:
def compare_void_catalogs(data_table_V1, data_table_V2, points_in_mask, U=1, V1_algorithm="VF", V2_algorithm="VF"):
    start_time = time.time()

    count_in_V1 = np.zeros(U)
    count_out_V1 = np.zeros(U)

    count_in_V2 = np.zeros(U)
    count_out_V2 = np.zeros(U)

    inside_both = np.zeros(U)
    inside_neither = np.zeros(U)
    inside_V1_not_V2 = np.zeros(U)
    inside_V2_not_V1 = np.zeros(U)

    points_in_mask_copy = points_in_mask.copy()

    kdTree_V1 = kd_tree(data_table_V1)
    kdTree_V2 = kd_tree(data_table_V2)

    for i in range(U):

        # need to randomize to let the KDTree work efficiently (per Lorenzo) so we shift by 3

        delta = np.random.rand(3)

        points_in_mask_copy[0] = points_in_mask[0] + delta[0]
        points_in_mask_copy[1] = points_in_mask[1] + delta[1]
        points_in_mask_copy[2] = points_in_mask[2] + delta[2]

        if V1_algorithm == "V2":
            true_inside_V1 = point_query_V2(points_in_mask_copy, kdTree_V1, data_table_V1)
        else:
            true_inside_V1 = point_query_VF(points_in_mask_copy, kdTree_V1, data_table_V1)

        count_in_V1[i] = np.sum(true_inside_V1)

        # the "~" inverts the array, so we can use the inverse of the trues to find all the falses
        count_out_V1[i] = np.sum(~true_inside_V1)

        if V2_algorithm == "V2":
            true_inside_V2 = point_query_V2(points_in_mask_copy, kdTree_V2, data_table_V2)
        else:
            true_inside_V2 = point_query_VF(points_in_mask_copy, kdTree_V2, data_table_V2)
        
        count_in_V2[i] = np.sum(true_inside_V2)

        count_out_V2[i] = np.sum(~true_inside_V2)

        # number of points that are inside both A and B
        inside_A_and_B = np.logical_and(true_inside_V1, true_inside_V2)
        inside_both[i] = np.sum(inside_A_and_B)

        # number of points that are in neither A and B
        not_inside_A_and_B = np.logical_and(~true_inside_V1, ~true_inside_V2)
        inside_neither[i] = np.sum(not_inside_A_and_B)

        # number of points that are in A but not B
        inside_A = np.logical_and(true_inside_V1, ~true_inside_V2)
        inside_V1_not_V2[i] = np.sum(inside_A)

        # number of points that are not in A but are in B
        inside_B = np.logical_and(~true_inside_V1, true_inside_V2)
        inside_V2_not_V1[i] = np.sum(inside_B)
        
    n_points = points_in_mask.shape[0]

    return count_in_V1, count_out_V1, count_in_V2, count_out_V2, inside_both, inside_neither, inside_V1_not_V2, inside_V2_not_V1, n_points


In [64]:
def VoidOverlap(data_table_V1, data_table_V2, title1, title2, mask_file, galaxy_table=None, zone_table_V1=None, zone_table_V2=None, use_mask=True, V1_algorithm="VF", V2_algorithm="VF", cosmo1='Planck2018', cosmo2='Planck2018', pruning1=None, pruning2=None, verbose=False):


    # Read galaxy file if provided
    if galaxy_table is not None:
        galaxy_table = Table.read(galaxy_table, format='fits')
        galaxy_table.sort('NSAID')
    
    # Read zone files if provided
    if zone_table_V1 is not None:
        zone_table_V1 = Table.read(zone_table_V1, format='fits')

    if zone_table_V2 is not None:
        zone_table_V2 = Table.read(zone_table_V2, format='fits')
    
    data_table_V1 = Table.read(data_table_V1, format='fits')
    data_table_V2 = Table.read(data_table_V2, format='fits')
    
    max_sphere_size = 0
    
    ## filtering for desired cosmo & pruning
    
    data_table_V1 = filter_and_analyze(data_table_V1, cosmo1, pruning1)
    data_table_V2 = filter_and_analyze(data_table_V2, cosmo2, pruning2)
    
    # if V2 algorithm, need x,y,z coords for each galaxy
    
    if V1_algorithm == "V2":
        data_table_V1 = prep_V2_cat(data_table_V1, zone_table_V1, galaxy_table)
    else:
        # Note the maximum sphere size for a VoidFinder catalog
        max_sphere_size = np.maximum(np.max(data_table_V1["Rad"]), max_sphere_size)
        
    if V2_algorithm == "V2":
        data_table_V2 = prep_V2_cat(data_table_V2, zone_table_V2, galaxy_table)
    else:
        max_sphere_size = np.maximum(np.max(data_table_V2["Rad"]), max_sphere_size)

    if use_mask:
        mask, mask_resolution, dist_limits = load_mask(mask_file)
        
        rmin = dist_limits[0]
        rmax = dist_limits[1]
    
    # Calculating boundaries
    xmin, xmax, ymin, ymax, zmin, zmax = calc_volume_boundaries(data_table_V1, data_table_V2)
    
    # Generating grid with default spacing 1.0 Mpc
    
    pts = generate_grid_points(xmin, xmax, ymin, ymax, zmin, zmax)
    
    if use_mask:
        points_in_mask = points_in_mask_filter(pts, mask, mask_resolution, rmin, rmax)
        
    kdTree_V1 = kd_tree(data_table_V1)
    kdTree_V2 = kd_tree(data_table_V2)
    
    count_in_V1, count_out_V1, count_in_V2, count_out_V2, inside_both, inside_neither, inside_V1_not_V2, inside_V2_not_V1, n_points = compare_void_catalogs(data_table_V1, data_table_V2, U=1, V1_algorithm=V1_algorithm, V2_algorithm=V2_algorithm, points_in_mask=points_in_mask)
    
    # displaying what was found in table format
    
    counts = {
    f'Count in {title1}': [count_in_V1[0]],
    f'Count out {title1}': [count_out_V1[0]],
    f'Count in {title2}': [count_in_V2[0]],
    f'Count out {title2}': [count_out_V2[0]],
    'Inside Both': [inside_both[0]],
    'Not Inside Both': [inside_neither[0]],
    f'Inside {title1}, not in {title2}': [inside_V1_not_V2[0]],
    f'Inside {title2}, not in {title2}': [inside_V2_not_V1[0]],
    'Total Points': [n_points]
    }
    
    if verbose:
        table_counts = Table()
        for category, count in counts.items():
            table_counts[category] = count
        table_counts_t = transpose_table(table_counts, id_col_name=f'Count in {title1}')
        print("Counts:")
        Table.max_rows = None
        Table.max_cols = None
        display(table_counts_t)
    
    
    # fractions for ratios
    
    r_V1 = count_in_V1[0] / n_points
    r_V2 = count_in_V2[0] / n_points
    r_V1_V2 = inside_both[0] / n_points
    r_not_V1_V2 = inside_neither[0] / n_points
    r_V1_not_V2 = inside_V1_not_V2[0] / n_points
    r_V2_not_V1 = inside_V2_not_V1[0] / n_points
    
    ratios = {
            f'Ratio {title1}': [r_V1],
            f'Ratio {title2}': [r_V2],
            f'Ratio {title1} and {title2}': [r_V1_V2],
            f'Ratio Not {title1} and {title2}': [r_not_V1_V2],
            f'Ratio {title1} Not {title2}': [r_V1_not_V2],
            f'Ratio {title2} Not {title1}': [r_V2_not_V1]
        }
    
    if verbose:
        table_ratios = Table()
        for category, ratio in ratios.items():
            table_ratios[category] = ratio
        table_ratios_t = transpose_table(table_ratios, id_col_name=f'Ratio {title1}')
        print("\nRatios:")
        display(table_ratios_t)

In [58]:
## File names

'NSA_main_mask.pickle'
"table1_VF_MaximalSpheres_2Cosmologies.dat.fits"
"table2_VF_AllSpheres_2Cosmologies.dat.gz.fits"
"table3_V2_Zobovoids_2Prunings_2Cosmologies.dat.gz.fits"
"table4_V2_Zones_2Prunings_2Cosmologies.dat.fits"
"table5_V2_Galaxies_2Prunings_2Cosmologies.dat.gz.fits"

## Cosmo names
'Planck2018'
'WMAP5     '

## Pruning names
'REVOLVER'
'VIDE    '


data_table_V1 = "table1_VF_MaximalSpheres_2Cosmologies.dat.fits"
data_table_V2 = "table1_VF_MaximalSpheres_2Cosmologies.dat.fits"
title1 = 'VFMaximalSpheresPlanck2018'
title2 = 'VFMaximalSpheresPlanck2018(1)'
mask_file = 'NSA_main_mask.pickle'
V1_algorithm = 'VF'
V2_algorithm = 'VF'
cosmo1 = 'Planck2018'
cosmo2 = 'Planck2018'

VoidOverlap(data_table_V1, data_table_V2, title1, title2, mask_file, galaxy_table=None, zone_table_V1=None, zone_table_V2=None, use_mask=True, V1_algorithm="VF", V2_algorithm="VF", cosmo1='Planck2018', cosmo2='Planck2018', pruning1=None, pruning2=None, verbose=True)


Counts:


Count in VFMaximalSpheresPlanck2018,9275042.0
str74,float64
Count out VFMaximalSpheresPlanck2018,17831412.0
Count in VFMaximalSpheresPlanck2018(1),9275042.0
Count out VFMaximalSpheresPlanck2018(1),17831412.0
Inside Both,9275042.0
Not Inside Both,17831412.0
"Inside VFMaximalSpheresPlanck2018, not in VFMaximalSpheresPlanck2018(1)",0.0
"Inside VFMaximalSpheresPlanck2018(1), not in VFMaximalSpheresPlanck2018(1)",0.0
Total Points,27106454.0



Ratios:


Ratio VFMaximalSpheresPlanck2018,0.3421709826006751
str70,float64
Ratio VFMaximalSpheresPlanck2018(1),0.3421709826006751
Ratio VFMaximalSpheresPlanck2018 and VFMaximalSpheresPlanck2018(1),0.3421709826006751
Ratio Not VFMaximalSpheresPlanck2018 and VFMaximalSpheresPlanck2018(1),0.6578290173993249
Ratio VFMaximalSpheresPlanck2018 Not VFMaximalSpheresPlanck2018(1),0.0
Ratio VFMaximalSpheresPlanck2018(1) Not VFMaximalSpheresPlanck2018,0.0


In [54]:
## File names

'NSA_main_mask.pickle'
"nsa_v1_0_1.fits"
"table1_VF_MaximalSpheres_2Cosmologies.dat.fits"
"table2_VF_AllSpheres_2Cosmologies.dat.gz.fits"
"table3_V2_Zobovoids_2Prunings_2Cosmologies.dat.gz.fits"
"table4_V2_Zones_2Prunings_2Cosmologies.dat.fits"
"table5_V2_Galaxies_2Prunings_2Cosmologies.dat.gz.fits"

## Cosmo names
'Planck2018'
'WMAP5     '

## Pruning names
'REVOLVER'
'VIDE    '



data_table_V1 = "table5_V2_Galaxies_2Prunings_2Cosmologies.dat.gz.fits"
data_table_V2 = "table5_V2_Galaxies_2Prunings_2Cosmologies.dat.gz.fits"
title1 = 'V2Planck2018REVOLVER'
title2 = 'V2Planck2018REVOLVER(1)'
mask_file = 'NSA_main_mask.pickle'
galaxy_table = "nsa_v1_0_1.fits"
zone_table_V1 = "table4_V2_Zones_2Prunings_2Cosmologies.dat.fits"
zone_table_V2 = "table4_V2_Zones_2Prunings_2Cosmologies.dat.fits"
V1_algorithm = 'V2'
V2_algorithm = 'V2'
cosmo1 = 'Planck2018'
cosmo2 = 'Planck2018'
pruning1 = 'REVOLVER'
pruning2 = 'REVOLVER'


VoidOverlap(data_table_V1, data_table_V2, title1, title2, mask_file, "nsa_v1_0_1.fits", "table4_V2_Zones_2Prunings_2Cosmologies.dat.fits", "table4_V2_Zones_2Prunings_2Cosmologies.dat.fits", use_mask=True, V1_algorithm=V1_algorithm, V2_algorithm=V2_algorithm, cosmo1=cosmo1, cosmo2=cosmo2, pruning1=pruning1, pruning2=pruning2, verbose=True)


Counts:


Count in V2Planck2018REVOLVER,20913796.0
str62,float64
Count out V2Planck2018REVOLVER,6799471.0
Count in V2Planck2018REVOLVER(1),20913796.0
Count out V2Planck2018REVOLVER(1),6799471.0
Inside Both,20913796.0
Not Inside Both,6799471.0
"Inside V2Planck2018REVOLVER, not in V2Planck2018REVOLVER(1)",0.0
"Inside V2Planck2018REVOLVER(1), not in V2Planck2018REVOLVER(1)",0.0
Total Points,27713267.0



Ratios:


Ratio V2Planck2018REVOLVER,0.7546492443492858
str58,float64
Ratio V2Planck2018REVOLVER(1),0.7546492443492858
Ratio V2Planck2018REVOLVER and V2Planck2018REVOLVER(1),0.7546492443492858
Ratio Not V2Planck2018REVOLVER and V2Planck2018REVOLVER(1),0.24535075565071415
Ratio V2Planck2018REVOLVER Not V2Planck2018REVOLVER(1),0.0
Ratio V2Planck2018REVOLVER(1) Not V2Planck2018REVOLVER,0.0


In [65]:
## File names

'NSA_main_mask.pickle'
"nsa_v1_0_1.fits"
"table1_VF_MaximalSpheres_2Cosmologies.dat.fits"
"table2_VF_AllSpheres_2Cosmologies.dat.gz.fits"
"table3_V2_Zobovoids_2Prunings_2Cosmologies.dat.gz.fits"
"table4_V2_Zones_2Prunings_2Cosmologies.dat.fits"
"table5_V2_Galaxies_2Prunings_2Cosmologies.dat.gz.fits"

## Cosmo names
'Planck2018'
'WMAP5     '

## Pruning names
'REVOLVER'
'VIDE    '



data_table_V1 = "table1_VF_MaximalSpheres_2Cosmologies.dat.fits"
data_table_V2 = "table5_V2_Galaxies_2Prunings_2Cosmologies.dat.gz.fits"
title1 = 'VFMaximalSpheresPlanck2018'
title2 = 'V2Planck2018REVOLVER'
mask_file = 'NSA_main_mask.pickle'
galaxy_table = "nsa_v1_0_1.fits"
zone_table_V2 = "table4_V2_Zones_2Prunings_2Cosmologies.dat.fits"
V1_algorithm = 'VF'
V2_algorithm = 'V2'
cosmo1 = 'Planck2018'
cosmo2 = 'Planck2018'
pruning2 = 'REVOLVER'


VoidOverlap('table1_VF_MaximalSpheres_2Cosmologies.dat.fits', 'table5_V2_Galaxies_2Prunings_2Cosmologies.dat.gz.fits', 'VFMaximalSpheresPlanck2018', 'V2Planck2018REVOLVER', 'NSA_main_mask.pickle', 'nsa_v1_0_1.fits', None, 'table4_V2_Zones_2Prunings_2Cosmologies.dat.fits', True, 'VF', 'V2', 'Planck2018', 'Planck2018', None, 'REVOLVER', True)


Counts:


Count in VFMaximalSpheresPlanck2018,9361464.0
str62,float64
Count out VFMaximalSpheresPlanck2018,18351803.0
Count in V2Planck2018REVOLVER,20913796.0
Count out V2Planck2018REVOLVER,6799471.0
Inside Both,7118339.0
Not Inside Both,4556346.0
"Inside VFMaximalSpheresPlanck2018, not in V2Planck2018REVOLVER",2243125.0
"Inside V2Planck2018REVOLVER, not in V2Planck2018REVOLVER",13795457.0
Total Points,27713267.0



Ratios:


Ratio VFMaximalSpheresPlanck2018,0.3377972001640947
str61,float64
Ratio V2Planck2018REVOLVER,0.7546492443492858
Ratio VFMaximalSpheresPlanck2018 and V2Planck2018REVOLVER,0.2568567249758031
Ratio Not VFMaximalSpheresPlanck2018 and V2Planck2018REVOLVER,0.16441028046242256
Ratio VFMaximalSpheresPlanck2018 Not V2Planck2018REVOLVER,0.08094047518829159
Ratio V2Planck2018REVOLVER Not VFMaximalSpheresPlanck2018,0.49779251937348273
